# Рекомендательная система SVD++: Funk MF (Matrix Factorization)

[Netflix Prize](https://ru.wikipedia.org/wiki/Netflix_Prize) - одно из самых известных соревнований в области машинного обучения.
Целью было создание алгоритма, который улучшал бы рекомендации фильмов и телешоу на 10%.

Один из наиболее успешных алгоритмов был алгоритм **SVD (Singular Value Decomposition)**, разработанный Саймоном Фанком и его коллегами.

Введем обозначения:
* $ R $ - разряженная [user-item matrix](https://habr.com/ru/articles/751470/#:~:text=User%2Ditem%20matrix,%D0%B8%20%D1%80%D0%B0%D0%B7%D1%80%D1%8F%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F%20%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0.), $ r_{ui} $ - элемент обучающей матрицы
* $ \hat R$ - прогнозируемые рейтинги, $ \hat r_{ui} $ - элемент прогнозируемой матрицы
* $ P $ - скрытые факторы пользователя (**user**), $ p_u $ - $u$-ая строка
* $ Q $ - скрытые факторы элемента (**item**), $ q_i $ - $i$-ый столбец
* $ \hat r_{ui} = q^T_i p_u $

Минимизируем квадратичную ошибку:

$$
\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda \left(||q_i||^2 + ||p_u||^2\right)
$$

$ \lambda $ - некоторый коэффициент

Но, при прогнозировании оценок необходимо учитывать **предубеждения** пользователей. 

Например, пользователь поставит более завышенную оценку фильму, если фильм признан сообществом. 
Менее популярный фильм получит более умеренную оценку.

Или, можно разделить пользователей на более и менее критичных. 
Кто-то легко ставит высокие рейтинги, кто-то - наоборот.

Эти смещения (предубеждения) необходимо учитывать при прогнозировании. 

Введем три переменные:
* $ \mu $ - среднее значение баллов, выставленных по всем items всеми users
* $ b_i $ - смещение, вносимое элементом (отклонение $q_i$ от $\mu$)
* $ b_u $ - предвзятость, внесенная пользователем ( отклонение $p_u$ от $\mu$)

Скорректированный прогноз с учетом смещения:

$$
\hat{r}_{ui} = \mu + b_i + b_u + q^T_i p_u 
$$


Обучение сводится к минимизации следующего выражения:

\begin{split}
&\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \mu + b_i + b_u + q^T_i p_u  \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right) = \\
= &\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right) 
\end{split}

Ошибку обозначим $ e_{ui} = r_{ui} - \hat{r}_{ui}$

Градиенты по $b_u$, $b_i$, $p_u$, $q_i$:

\begin{split}
&\nabla b_u = \left(r_{ui} - \mu + b_i + b_u + q^T_i p_u  \right) - \lambda b_u  = e_{ui} - \lambda b_u \\
&\nabla b_i = e_{ui} - \lambda b_i \\
&\nabla p_u = e_{ui} \cdot q_i - \lambda p_u \\
&\nabla q_i = e_{ui} \cdot p_u - \lambda q_i
\end{split}

Тогда градиентный шаг выполняется по формулам: 

\begin{split}
& b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)\\
& b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)\\
& p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)\\
& q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)
\end{split}

$ \gamma $ - некоторый коэффициент

Реализуем Funk MF алгоритм. 

<!-- Используем [Movielens Latest Dataset](https://www.kaggle.com/datasets/deepak1011/movielens-latest-datasets). -->

In [1]:
import numpy as np
import pandas as pd

In [2]:
k = 10 # максимальная оценка

movies = ['Фантазия', 'ВАЛЛ-И', 'Пиноккио', 'Бемби' , 'Шрэк', 'Дамбо', 'Спасатели', 'Геркулес', 'Кунг-фу Панда']
m_movies = len(movies)

users = ['Андрей', 'Аня', 'Алиса', 'Ваня', 'Леша', 'Оксана', 'Саша', 'Паша', 'Сеня', 'Гриша']        
n_users = len(users)

In [3]:
# Инициализируем R_train

np.random.seed(42)

N = np.random.randint(50, 60) # сколько оценок будет поставлено

ind_users, ind_movies, rating = [], [], []
user_movie = [] # чтобы пара user-movie не повторялись

for _ in range(N):
    user = np.random.randint(0, n_users)
    movie = np.random.randint(0, m_movies)
    if not [user, movie] in user_movie:
        ind_users.append(user)
        ind_movies.append(movie)
        rating.append(np.random.randint(1, k)) # случайная оценка пользователя фильму
        user_movie.append([user, movie])        
N = len(user_movie)

data = {'userId': ind_users, 'movieId': ind_movies, 'rating': rating}
R_train = pd.DataFrame(data = data)
R_train.head(3)

,userId,movieId,rating
0,3,7,5
1,6,2,7
2,7,4,4


In [4]:
# Посмотри на User-item matrix

User_item_matrix = R_train.pivot(columns = 'movieId', index = 'userId', values = 'rating')
User_item_matrix

movieId,0,1,2,3,4,5,6,7,8
userId,,,,,,,,,
0,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
1,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
2,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
4,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
5,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
6,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
7,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
8,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0


In [5]:
User_item_matrix.rename(columns = dict(zip(User_item_matrix.columns, movies)), inplace = True)
User_item_matrix.set_index(pd.Index(users), inplace=True)
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0


In [6]:
# среднее по всем рейтингам
mu = R_train['rating'].mean()
mu

5.441860465116279

In [7]:
# инициализируем смещение, вносимое фильмами и "предвзятость" пользователей

bu = np.zeros(n_users)
bm = np.zeros(m_movies)

print(bu.shape, bm.shape)

(10,) (9,)


In [8]:
# инициализируем скрытые факторы пользователей и скрытые факторы фильмов

d = 5 # главных компонент

pu = np.random.normal(0, 0.1, (n_users, d))
qm = np.random.normal(0, 0.1, (m_movies, d))

print(pu.shape, qm.shape)

(10, 5) (9, 5)


In [9]:
# Градиентный спуск

epoch = 5
gamma = 0.02
lmbda = 0.03

for _ in range(epoch):
    
    for ind in range(R_train.shape[0]):

        u = R_train['userId'][ind] 
        m = R_train['movieId'][ind] 
        r = R_train['rating'][ind]

        err = r - (mu + bu[u] + bm[m] + qm[m] @ pu[u])

        bu[u] += gamma * (err - lmbda * bu[u]) 
        bm[m] += gamma * (err - lmbda * bm[m]) 
        
        pu[u] += gamma * (err * qm[m] - lmbda * pu[u])
        qm[m] += gamma * (err * pu[u] - lmbda * qm[m])

In [10]:
R_pred = np.zeros((n_users, m_movies))

for u in range(n_users):
    for m in range(m_movies):
        R_pred[u][m] = round(mu + bu[u] + bm[m] + qm[m] @ pu[u], 2)

pd.DataFrame(R_pred, users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,4.78,5.93,5.61,5.70,5.10,5.41,6.54,5.26,6.03
Аня,4.88,5.97,5.70,5.81,5.20,5.50,6.66,5.33,6.15
Алиса,4.41,5.57,5.43,5.53,4.60,5.16,6.17,4.91,5.83
Ваня,4.40,5.54,5.36,5.45,4.74,5.11,6.20,4.90,5.72
Леша,4.71,5.80,5.51,5.59,5.05,5.29,6.49,5.17,5.93
Оксана,4.99,6.23,5.92,6.00,5.20,5.70,6.71,5.52,6.34
Саша,5.08,6.27,6.10,6.23,5.54,5.85,6.92,5.59,6.45
Паша,4.18,5.28,4.97,5.08,4.46,4.79,5.94,4.63,5.45
Сеня,4.03,5.24,5.05,5.09,4.26,4.76,5.78,4.57,5.38
Гриша,4.94,6.12,5.84,5.96,5.16,5.65,6.67,5.43,6.30


In [11]:
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0


Можно воспользоваться библиотекой [scikit-surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD).

In [12]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [13]:
# создание объекта класса Reader
reader = Reader(rating_scale=(1, 10))

# создание объекта класса Dataset
dataset = Dataset.load_from_df(R_train[['userId', 'movieId', 'rating']], reader)

# разбиение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(dataset, test_size = 0.1)

# создание экземпляра класса SVD
model = SVD()

# обучение модели на обучающей выборке
model.fit(trainset)

# предсказание рейтингов на тестовой выборке
predictions = model.test(testset)

# оценка качества модели
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

RMSE: 2.1303
RMSE: 2.1302702798661457
MAE:  1.6641
MAE: 1.664104984744074


In [14]:
R_pred_surprise = np.zeros((n_users, m_movies))

for u in range(n_users):
    for m in range(m_movies):
        R_pred_surprise[u][m] = model.predict(u, m).est
        
pd.DataFrame(np.round(R_pred_surprise, 2), users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,4.96,6.08,5.31,5.54,5.70,5.36,6.39,5.38,6.24
Аня,5.31,5.25,5.52,5.90,6.18,5.39,6.32,5.19,6.08
Алиса,4.27,5.55,5.13,5.54,4.38,5.06,6.13,5.07,5.74
Ваня,3.78,5.70,5.31,5.53,4.54,5.20,6.51,5.18,5.69
Леша,4.93,5.87,5.34,5.82,5.03,5.04,6.59,5.41,6.01
Оксана,5.06,6.99,5.76,5.82,4.24,5.62,6.51,6.43,6.31
Саша,4.32,5.67,6.14,6.86,6.37,6.38,6.71,5.62,6.87
Паша,4.46,5.41,4.57,5.19,4.80,4.26,6.69,4.54,5.50
Сеня,3.86,6.38,5.21,5.15,3.47,5.03,6.25,5.20,5.38
Гриша,5.00,6.41,5.57,5.93,4.35,5.49,7.21,5.77,6.71


In [15]:
pd.DataFrame(R_pred, users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,4.78,5.93,5.61,5.70,5.10,5.41,6.54,5.26,6.03
Аня,4.88,5.97,5.70,5.81,5.20,5.50,6.66,5.33,6.15
Алиса,4.41,5.57,5.43,5.53,4.60,5.16,6.17,4.91,5.83
Ваня,4.40,5.54,5.36,5.45,4.74,5.11,6.20,4.90,5.72
Леша,4.71,5.80,5.51,5.59,5.05,5.29,6.49,5.17,5.93
Оксана,4.99,6.23,5.92,6.00,5.20,5.70,6.71,5.52,6.34
Саша,5.08,6.27,6.10,6.23,5.54,5.85,6.92,5.59,6.45
Паша,4.18,5.28,4.97,5.08,4.46,4.79,5.94,4.63,5.45
Сеня,4.03,5.24,5.05,5.09,4.26,4.76,5.78,4.57,5.38
Гриша,4.94,6.12,5.84,5.96,5.16,5.65,6.67,5.43,6.30


In [16]:
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0
